# Logistic Regression: Implement signle layer perceptron for glass classification

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

torch.manual_seed(67)
np.random.seed(67)

# Load dataset
df = pd.read_csv(r'inputs\glass.csv')

print("Glass Classification - Single Layer Perceptron (PyTorch)")
print(f"\nDataset: {df.shape[0]} samples, {df.shape[1]-1} features")
print(f"\nClass counts:\n{df['Type'].value_counts().sort_index()}")

# Extract features and labels
X = df.drop('Type', axis=1).values
y = df['Type'].values

# Remap labels to consecutive integers (0, 1, 2, ...)
encode = LabelEncoder()
y = encode.fit_transform(y)

n_class = len(np.unique(y))
print(f"\nFeatures: {X.shape}, Classes: {n_class}")
print(f"Label mapping: {dict(zip(encode.classes_, range(len(encode.classes_))))}")

# Split and scale
x_tr, x_ts, y_tr, y_ts = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

scale = StandardScaler()
x_tr = scale.fit_transform(x_tr)
x_ts = scale.transform(x_ts)

print(f"Train: {x_tr.shape[0]}, Test: {x_ts.shape[0]}")

# Convert to tensors
x_tr_t = torch.FloatTensor(x_tr)
y_tr_t = torch.LongTensor(y_tr)
x_ts_t = torch.FloatTensor(x_ts)
y_ts_t = torch.LongTensor(y_ts)

# Data loaders
batch = 16
tr_data = TensorDataset(x_tr_t, y_tr_t)
ts_data = TensorDataset(x_ts_t, y_ts_t)
tr_load = DataLoader(tr_data, batch_size=batch, shuffle=True)
ts_load = DataLoader(ts_data, batch_size=batch, shuffle=False)


class SLP(nn.Module):
    def __init__(self, n_feat, n_class):
        super(SLP, self).__init__()
        self.layer = nn.Linear(n_feat, n_class)

    def forward(self, x):
        return self.layer(x)


# Initialize
n_feat = x_tr.shape[1]
model = SLP(n_feat, n_class)
criter = nn.CrossEntropyLoss()
optim_fn = optim.SGD(model.parameters(), lr=0.1)

print("Architecture")
print(model)
n_param = sum(p.numel() for p in model.parameters())
print(f"\nParams: {n_param}")


def train(model, tr_load, ts_load, criter, optim_fn, epochs=1000):
    hist_loss = []
    hist_tr = []
    hist_ts = []

    for ep in range(epochs):
        model.train()
        loss_sum = 0.0
        correct = 0
        total = 0

        for inputs, labels in tr_load:
            outputs = model(inputs)
            loss = criter(outputs, labels)

            optim_fn.zero_grad()
            loss.backward()
            optim_fn.step()

            loss_sum += loss.item()
            _, pred = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (pred == labels).sum().item()

        avg_loss = loss_sum / len(tr_load)
        tr_acc = correct / total
        hist_loss.append(avg_loss)
        hist_tr.append(tr_acc)

        model.eval()
        ts_cor = 0
        ts_tot = 0

        with torch.no_grad():
            for inputs, labels in ts_load:
                outputs = model(inputs)
                _, pred = torch.max(outputs.data, 1)
                ts_tot += labels.size(0)
                ts_cor += (pred == labels).sum().item()

        ts_acc = ts_cor / ts_tot
        hist_ts.append(ts_acc)

        if (ep + 1) % 10 == 0:
            print(f"Epoch {ep+1:4d} | Loss: {avg_loss:.4f} | "
                  f"Train: {tr_acc:.4f} | Test: {ts_acc:.4f}")

    return hist_loss, hist_tr, hist_ts


# Train
hist_loss, hist_tr, hist_ts = train(
    model, tr_load, ts_load, criter, optim_fn, epochs=1000
)

# Evaluate
print("Results")
def get_pred(model, loader):
    model.eval()
    preds = []
    labels = []

    with torch.no_grad():
        for inputs, labs in loader:
            outputs = model(inputs)
            _, pred = torch.max(outputs.data, 1)
            preds.extend(pred.cpu().numpy())
            labels.extend(labs.cpu().numpy())

    return np.array(preds), np.array(labels)


y_tr_p, y_tr_t = get_pred(model, tr_load)
y_ts_p, y_ts_t = get_pred(model, ts_load)

tr_acc = accuracy_score(y_tr_t, y_tr_p)
ts_acc = accuracy_score(y_ts_t, y_ts_p)

print(f"\nTrain acc: {tr_acc:.4f} ({tr_acc*100:.2f}%)")
print(f"Test acc: {ts_acc:.4f} ({ts_acc*100:.2f}%)")


print("Report (Test)")
print(classification_report(y_ts_t, y_ts_p, target_names=[str(c) for c in encode.classes_]))

print("\nConfusion Matrix")
cm = confusion_matrix(y_ts_t, y_ts_p)
print(cm)

# Plots
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].plot(hist_loss)
axes[0].set_title('Loss vs Epochs')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Cross-Entropy')
axes[0].grid(True)

axes[1].plot(hist_tr, label='Train')
axes[1].plot(hist_ts, label='Test')
axes[1].set_title('Accuracy vs Epochs')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig(r'outputs\metrics_pytorch.png', dpi=300, bbox_inches='tight')
plt.close()

torch.save(model.state_dict(), r'outputs\glass_model.pth')

print("\nGraph saved: metrics_pytorch.png")
print("Model saved: glass_model.pth")

Glass Classification - Single Layer Perceptron (PyTorch)

Dataset: 214 samples, 9 features

Class counts:
Type
1    70
2    76
3    17
5    13
6     9
7    29
Name: count, dtype: int64

Features: (214, 9), Classes: 6
Label mapping: {np.int64(1): 0, np.int64(2): 1, np.int64(3): 2, np.int64(5): 3, np.int64(6): 4, np.int64(7): 5}
Train: 149, Test: 65

Architecture
SLP(
  (layer): Linear(in_features=9, out_features=6, bias=True)
)

Params: 60

Training
Epoch   10 | Loss: 0.9879 | Train: 0.6107 | Test: 0.6923
Epoch   20 | Loss: 0.9172 | Train: 0.6376 | Test: 0.7538
Epoch   30 | Loss: 0.8675 | Train: 0.6309 | Test: 0.7385
Epoch   40 | Loss: 0.8022 | Train: 0.6510 | Test: 0.7385
Epoch   50 | Loss: 0.8336 | Train: 0.6510 | Test: 0.7385
Epoch   60 | Loss: 0.7757 | Train: 0.6376 | Test: 0.7385
Epoch   70 | Loss: 0.7960 | Train: 0.6711 | Test: 0.7385
Epoch   80 | Loss: 0.8150 | Train: 0.6577 | Test: 0.7385
Epoch   90 | Loss: 0.7786 | Train: 0.6711 | Test: 0.7385
Epoch  100 | Loss: 0.7604 | Train:

C:\MyVirtualEnvs\Pytorch_NN_RnD\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\MyVirtualEnvs\Pytorch_NN_RnD\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\MyVirtualEnvs\Pytorch_NN_RnD\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])



Graph saved: metrics_pytorch.png
Model saved: glass_model.pth
